<a href="https://colab.research.google.com/github/leyli16/HousingPricePrediction/blob/main/final_project_merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Packages

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

## Loading data through kaggle setup

In [8]:
%%capture
!pip instsall kaggle

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


# USA Real Estate Data Wrangling and Cleaning

## Loading in USA Real Estate Dataset


In [11]:
#!/bin/bash
!!kaggle datasets download ahmedshahriarsakib/usa-real-estate-dataset

['Dataset URL: https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset',
 'License(s): other',
 'usa-real-estate-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)']

In [12]:
!unzip usa-real-estate-dataset.zip

Archive:  usa-real-estate-dataset.zip
  inflating: realtor-data.zip.csv    


In [13]:
df_raw = pd.read_csv('realtor-data.zip.csv')
df_raw.head(10)

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN
5,103378.0,for_sale,179000.0,4.0,3.0,0.46,1850806.0,San Sebastian,Puerto Rico,612.0,2520.0,NaN
6,1205.0,for_sale,50000.0,3.0,1.0,0.20,1298094.0,Ciales,Puerto Rico,639.0,2040.0,NaN
7,50739.0,for_sale,71600.0,3.0,2.0,0.08,1048466.0,Ponce,Puerto Rico,731.0,1050.0,NaN
8,81909.0,for_sale,100000.0,2.0,1.0,0.09,734904.0,Ponce,Puerto Rico,730.0,1092.0,NaN
9,65672.0,for_sale,300000.0,5.0,3.0,7.46,1946226.0,Las Marias,Puerto Rico,670.0,5403.0,NaN


In [14]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zip_code        float64
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(8), object(4)
memory usage: 203.8+ MB


In [15]:
df_raw.describe()

,brokered_by,price,bed,bath,acre_lot,street,zip_code,house_size
count,2.221849e+06,2.224841e+06,1.745065e+06,1.714611e+06,1.900793e+06,2.215516e+06,2.226083e+06,1.657898e+06
mean,5.293989e+04,5.241955e+05,3.275841e+00,2.496440e+00,1.522303e+01,1.012325e+06,5.218668e+04,2.714471e+03
std,3.064275e+04,2.138893e+06,1.567274e+00,1.652573e+00,7.628238e+02,5.837635e+05,2.895408e+04,8.081635e+05
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
25%,2.386100e+04,1.650000e+05,3.000000e+00,2.000000e+00,1.500000e-01,5.063128e+05,2.961700e+04,1.300000e+03
50%,5.288400e+04,3.250000e+05,3.000000e+00,2.000000e+00,2.600000e-01,1.012766e+06,4.838200e+04,1.760000e+03
75%,7.918300e+04,5.500000e+05,4.000000e+00,3.000000e+00,9.800000e-01,1.521173e+06,7.807000e+04,2.413000e+03
max,1.101420e+05,2.147484e+09,4.730000e+02,8.300000e+02,1.000000e+05,2.001357e+06,9.999900e+04,1.040400e+09


In [16]:
df_raw.describe(include = 'object')

,status,city,state,prev_sold_date
count,2226382,2224975,2226374,1492085
unique,3,20098,55,14954
top,for_sale,Houston,Florida,2022-03-31
freq,1389306,23862,249432,17171


## Cleaning up USA Real Estate data

In [17]:
df_raw.duplicated().sum()

np.int64(0)

In [18]:
# Keep only the necessary columns
df_req_cols = df_raw [['price', 'bed', 'bath', 'acre_lot', 'city', 'state', 'zip_code', 'house_size', 'status']]

In [19]:
total_missing = df_req_cols.isna().sum()*100/len(df_req_cols)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
price,0.069215
bed,21.618797
bath,22.986666
acre_lot,14.624130
city,0.063197
state,0.000359
zip_code,0.013430
house_size,25.533983
status,0.000000


In [20]:
# Drop nulls, reset and drop the index
df_nadropped = df_req_cols.dropna().reset_index(drop=True)
df_nadropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360347 entries, 0 to 1360346
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   price       1360347 non-null  float64
 1   bed         1360347 non-null  float64
 2   bath        1360347 non-null  float64
 3   acre_lot    1360347 non-null  float64
 4   city        1360347 non-null  object 
 5   state       1360347 non-null  object 
 6   zip_code    1360347 non-null  float64
 7   house_size  1360347 non-null  float64
 8   status      1360347 non-null  object 
dtypes: float64(6), object(3)
memory usage: 93.4+ MB


In [21]:
total_missing = df_nadropped.isna().sum()*100/len(df_nadropped)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
price,0.0
bed,0.0
bath,0.0
acre_lot,0.0
city,0.0
state,0.0
zip_code,0.0
house_size,0.0
status,0.0


In [22]:
# cast date types
df_typed = df_nadropped.astype({'price': 'float', 'bed': 'int', 'bath': 'int', 'acre_lot': 'float', 'city':'str', 'state':'str', 'zip_code':'int', 'house_size': 'float'})

In [23]:
df_typed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360347 entries, 0 to 1360346
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   price       1360347 non-null  float64
 1   bed         1360347 non-null  int64  
 2   bath        1360347 non-null  int64  
 3   acre_lot    1360347 non-null  float64
 4   city        1360347 non-null  object 
 5   state       1360347 non-null  object 
 6   zip_code    1360347 non-null  int64  
 7   house_size  1360347 non-null  float64
 8   status      1360347 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 93.4+ MB


In [24]:
# Filter rows with status = sold
df_sold = df_typed[df_typed['status'] == 'sold'].drop(columns=['status'])

In [25]:
# Adding a column price_per_sqft
df_sold['price_per_sqft'] = df_sold['price'] / df_sold['house_size']

In [26]:
df_sold.head()

,price,bed,bath,acre_lot,city,state,zip_code,house_size,price_per_sqft
750602,524900.0,3,2,0.09,Aguada,Puerto Rico,602,2200.0,238.590909
750603,90000.0,3,2,0.08,Aguadilla,Puerto Rico,603,1421.0,63.335679
750604,22500.0,2,1,0.32,Anasco,Puerto Rico,610,850.0,26.470588
750605,168000.0,6,4,0.05,Anasco,Puerto Rico,610,3422.0,49.094097
750606,200000.0,3,1,4.98,Arecibo,Puerto Rico,612,580.0,344.827586


In [27]:
df_real_estate = df_sold

## Loading in Zip code to County Dataset

In [28]:
#!/bin/bash
!!kaggle datasets download danofer/zipcodes-county-fips-crosswalk

['Dataset URL: https://www.kaggle.com/datasets/danofer/zipcodes-county-fips-crosswalk',
 'License(s): CC0-1.0']

In [29]:
!unzip zipcodes-county-fips-crosswalk.zip

Archive:  zipcodes-county-fips-crosswalk.zip
  inflating: ZIP-COUNTY-FIPS_2017-06.csv  


In [30]:
zip_county_df = pd.read_csv('ZIP-COUNTY-FIPS_2017-06.csv')
zip_county_df.head(10)

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1
5,36701,Autauga County,AL,1001,H1
6,36091,Autauga County,AL,1001,H1
7,36051,Autauga County,AL,1001,H1
8,36068,Autauga County,AL,1001,H1
9,36008,Autauga County,AL,1001,H1


In [31]:
zip_county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52889 entries, 0 to 52888
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ZIP         52889 non-null  int64 
 1   COUNTYNAME  52889 non-null  object
 2   STATE       52889 non-null  object
 3   STCOUNTYFP  52889 non-null  int64 
 4   CLASSFP     52889 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


## Cleaning up Zip Code to County Dataset

In [32]:
zip_county_df.duplicated().sum()

np.int64(0)

In [33]:
total_missing = zip_county_df.isna().sum() * 100 / len(zip_county_df)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
ZIP,0.0
COUNTYNAME,0.0
STATE,0.0
STCOUNTYFP,0.0
CLASSFP,0.0


In [34]:
# cast date types
zip_county_df = zip_county_df.astype({'ZIP': 'int', 'COUNTYNAME':'str', 'STATE':'str'})

In [35]:
zip_county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52889 entries, 0 to 52888
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ZIP         52889 non-null  int64 
 1   COUNTYNAME  52889 non-null  object
 2   STATE       52889 non-null  object
 3   STCOUNTYFP  52889 non-null  int64 
 4   CLASSFP     52889 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


In [36]:
zip_county_df.rename(columns={'ZIP': 'zip_code', 'COUNTYNAME':'county', 'STATE':'state'}, inplace=True)

In [37]:
zip_county_df.drop(columns=['STCOUNTYFP', 'CLASSFP'], inplace=True)

In [38]:
# count unique counties per ZIP code
zip_counts = zip_county_df['zip_code'].value_counts()

# Step 1: Find ZIP codes with only one county
zips_one_county = zip_counts[zip_counts == 1].index
# Step 2: Filter out rows with those ZIP codes
zip_county_df = zip_county_df[zip_county_df['zip_code'].isin(zips_one_county)]

In [39]:
zip_county_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29102 entries, 0 to 52888
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   zip_code  29102 non-null  int64 
 1   county    29102 non-null  object
 2   state     29102 non-null  object
dtypes: int64(1), object(2)
memory usage: 909.4+ KB


## Create new column "county" in USA Real Estate Dataset

In [40]:
# Merge df with zip_county_df to get county info
real_estate_df = pd.merge(df_real_estate, zip_county_df, on='zip_code', how='inner')
real_estate_df.drop(columns=['state_x'], inplace=True)
real_estate_df.rename(columns={'state_y':'state'},inplace=True)
real_estate_df.head(10)

,price,bed,bath,acre_lot,city,zip_code,house_size,price_per_sqft,county,state
0,22500.0,2,1,0.32,Anasco,610,850.0,26.470588,Anasco Municipio,PR
1,168000.0,6,4,0.05,Anasco,610,3422.0,49.094097,Anasco Municipio,PR
2,120000.0,4,2,0.18,Sabana Grande,637,1188.0,101.010101,Sabana Grande Municipio,PR
3,133000.0,3,1,0.60,Hatillo,659,1049.0,126.787417,Hatillo Municipio,PR
4,177000.0,2,2,0.10,Isabela,662,4674.0,37.869063,Isabela Municipio,PR
5,220000.0,5,3,0.11,Isabela,662,2992.0,73.529412,Isabela Municipio,PR
6,175000.0,3,3,0.45,Las Marias,670,3090.0,56.634304,Las Marias Municipio,PR
7,149000.0,3,2,0.28,Mayaguez,682,2000.0,74.500000,Mayaguez Municipio,PR
8,2750000.0,3,4,0.15,San Juan,907,2820.0,975.177305,San Juan Municipio,PR
9,185000.0,3,2,13.47,Maunabo,707,1159.0,159.620362,Maunabo Municipio,PR


In [41]:
real_estate_df.shape

(451948, 10)

In [42]:
real_estate_df.describe()

,price,bed,bath,acre_lot,zip_code,house_size,price_per_sqft
count,4.519480e+05,451948.000000,451948.000000,451948.000000,451948.000000,451948.000000,451948.000000
mean,5.518665e+05,3.284531,2.428421,11.169715,59467.386436,1943.216633,285.079374
std,7.508103e+05,1.316137,1.195875,774.451696,31666.360915,1763.082967,240.109022
min,1.000000e+00,1.000000,1.000000,0.000000,610.000000,100.000000,0.000282
25%,2.499075e+05,3.000000,2.000000,0.130000,31312.000000,1303.000000,150.130548
50%,3.950000e+05,3.000000,2.000000,0.190000,70433.000000,1714.000000,214.868181
75%,6.250000e+05,4.000000,3.000000,0.320000,91106.000000,2300.000000,333.266667
max,8.000000e+07,444.000000,222.000000,100000.000000,99402.000000,927828.000000,15015.015015


In [43]:
real_estate_df.describe(include = 'object')

,city,county,state
count,451948,451948,451948
unique,8090,1267,51
top,Houston,Los Angeles County,CA
freq,10570,20836,93327


## Loading in Zip Code Demographics

In [44]:
#!/bin/bash
!!kaggle datasets download erdi28/zip-codes-demographics

['Dataset URL: https://www.kaggle.com/datasets/erdi28/zip-codes-demographics',
 'License(s): MIT']

In [45]:
!unzip zip-codes-demographics.zip

Archive:  zip-codes-demographics.zip
  inflating: zip_code_demographics.csv  


In [46]:
zip_code_demographics_df_raw = pd.read_csv('zip_code_demographics.csv')
zip_code_demographics_df_raw.head(10)

,zip,lat,lng,city,state_id,state_name,population,density,county_name,po_box,dist_highway,dist2_large_airport,dist2_medium_airport,dist_to_shore,number_of_business,adjusted_gross_income,total_income_amount,number_of_returns
0,1001,42.06262,-72.62521,Agawam,MA,Massachusetts,16088,550.1,Hampden,0,1.387035,106.145765,12.946212,93.049251,438.0,598807,6.047690e+05,9320
1,1002,42.37633,-72.46462,Amherst,MA,Massachusetts,27323,198.1,Hampshire,0,14.438177,112.264368,21.080079,133.370144,571.0,989558,1.005796e+06,9880
2,1005,42.42117,-72.10655,Barre,MA,Massachusetts,4947,44.2,Worcester,0,16.788339,90.664964,25.547718,97.639881,97.0,164207,1.660540e+05,2490
3,1007,42.28163,-72.40009,Belchertown,MA,Massachusetts,15304,107.7,Hampshire,0,13.663839,101.552921,14.762395,114.406034,217.0,647074,6.547390e+05,7970
4,1008,42.18234,-72.95819,Blandford,MA,Massachusetts,1171,7.4,Hampden,0,2.593655,136.548797,20.177950,107.466779,18.0,47826,4.824100e+04,660
5,1010,42.12904,-72.20597,Brimfield,MA,Massachusetts,3703,40.6,Hampden,0,4.737271,78.645500,28.049262,94.928024,73.0,155666,1.576770e+05,1980
6,1011,42.30233,-72.96448,Chester,MA,Massachusetts,1332,15.7,Hampden,0,8.058693,142.414627,26.041908,128.563170,13.0,38223,3.855300e+04,630
7,1012,42.38495,-72.84675,Chesterfield,MA,Massachusetts,503,16.1,Hampshire,0,18.331096,138.381684,27.466664,137.693285,10.0,24826,2.510200e+04,410
8,1013,42.16059,-72.60788,Chicopee,MA,Massachusetts,22709,1549.1,Hampden,0,0.062286,109.114246,7.075893,104.576258,317.0,516431,5.201740e+05,11270
9,1020,42.17618,-72.56538,Chicopee,MA,Massachusetts,30704,951.4,Hampden,0,0.925140,106.769095,3.205533,105.789421,550.0,828125,8.346140e+05,15760


In [47]:
zip_code_demographics_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33704 entries, 0 to 33703
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   zip                    33704 non-null  int64  
 1   lat                    33704 non-null  float64
 2   lng                    33704 non-null  float64
 3   city                   33704 non-null  object 
 4   state_id               33704 non-null  object 
 5   state_name             33704 non-null  object 
 6   population             33704 non-null  int64  
 7   density                33704 non-null  float64
 8   county_name            33704 non-null  object 
 9   po_box                 33704 non-null  int64  
 10  dist_highway           33704 non-null  float64
 11  dist2_large_airport    33704 non-null  float64
 12  dist2_medium_airport   33704 non-null  float64
 13  dist_to_shore          33704 non-null  float64
 14  number_of_business     33624 non-null  float64
 15  ad

In [48]:
zip_code_demographics_df_raw.describe()

,zip,lat,lng,population,density,po_box,dist_highway,dist2_large_airport,dist2_medium_airport,dist_to_shore,number_of_business,adjusted_gross_income,total_income_amount,number_of_returns
count,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33624.000000,3.370400e+04,3.370400e+04,33704.000000
mean,49842.607821,38.888195,-91.051366,11287.959738,528.704068,0.179949,29.291294,158.648048,40.854624,304.798736,260.694920,4.251341e+05,4.298943e+05,5341.873665
std,27451.111442,5.228075,15.460095,15200.216997,2219.277025,0.384151,174.104869,166.207042,80.224956,298.948686,432.112814,7.561519e+05,7.644416e+05,7273.613660
min,1001.000000,-14.219840,-176.629620,0.000000,0.000000,0.000000,0.000194,0.112768,0.134153,0.011449,1.000000,5.770000e+02,5.770000e+02,90.000000
25%,27052.750000,35.427243,-97.255342,1253.750000,8.200000,0.000000,3.050420,59.434074,19.513217,61.286177,19.000000,3.247500e+04,3.287925e+04,580.000000
50%,49780.500000,39.502970,-88.224820,4099.000000,29.000000,0.000000,11.541912,125.957938,35.131017,207.328858,74.000000,1.128665e+05,1.140365e+05,1880.000000
75%,72210.250000,42.110000,-80.289333,16113.250000,238.825000,0.000000,30.046597,209.476362,55.231087,471.857289,334.000000,5.052290e+05,5.092270e+05,7560.000000
max,99929.000000,71.274340,144.876370,130352.000000,68424.500000,1.000000,10767.713382,6127.454722,5101.697518,1336.551268,7263.000000,1.522467e+07,1.527798e+07,61920.000000


In [49]:
zip_code_demographics_df_raw.describe(include = 'object')

,city,state_id,state_name,county_name
count,33704,33704,33704,33704
unique,17551,51,51,1800
top,Houston,TX,Texas,Washington
freq,106,1990,1990,401


## Cleaning Zip Code Demographics Data

In [50]:
zip_code_demographics_df_raw.duplicated().sum()

np.int64(0)

In [51]:
# Keep only the necessary columns
zip_code_demographics_df = zip_code_demographics_df_raw [['zip', 'population', 'density', 'dist_highway', 'dist2_large_airport', 'dist_to_shore', 'number_of_business', 'adjusted_gross_income']]

In [52]:
total_missing = zip_code_demographics_df.isna().sum() * 100 / len(zip_code_demographics_df)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
zip,0.000000
population,0.000000
density,0.000000
dist_highway,0.000000
dist2_large_airport,0.000000
dist_to_shore,0.000000
number_of_business,0.237361
adjusted_gross_income,0.000000


In [53]:
# Drop nulls
zip_code_demographics_df = zip_code_demographics_df.dropna()
# Reset and drop the index
zip_code_demographics_df = zip_code_demographics_df.reset_index(drop=True)

In [54]:
total_missing = zip_code_demographics_df.isna().sum() * 100 / len(zip_code_demographics_df)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
zip,0.0
population,0.0
density,0.0
dist_highway,0.0
dist2_large_airport,0.0
dist_to_shore,0.0
number_of_business,0.0
adjusted_gross_income,0.0


In [55]:
# cast date types
zip_code_demographics_df = zip_code_demographics_df.astype({'zip': 'int', 'population': 'float', 'density': 'float', 'dist_highway': 'float', 'dist2_large_airport': 'float', 'dist_to_shore': 'float', 'number_of_business': 'float', 'adjusted_gross_income': 'float'})

In [56]:
zip_code_demographics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33624 entries, 0 to 33623
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   zip                    33624 non-null  int64  
 1   population             33624 non-null  float64
 2   density                33624 non-null  float64
 3   dist_highway           33624 non-null  float64
 4   dist2_large_airport    33624 non-null  float64
 5   dist_to_shore          33624 non-null  float64
 6   number_of_business     33624 non-null  float64
 7   adjusted_gross_income  33624 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 2.1 MB


In [57]:
zip_code_demographics_df.rename(columns={'zip': 'zip_code'}, inplace=True)

## Merge real_estate_df with zip_code_demographics_df to add the zip code related features

In [58]:
real_estate_features_df = pd.merge(real_estate_df, zip_code_demographics_df, on='zip_code', how='inner')
real_estate_features_df.head(10)

,price,bed,bath,acre_lot,city,zip_code,house_size,price_per_sqft,county,state,population,density,dist_highway,dist2_large_airport,dist_to_shore,number_of_business,adjusted_gross_income
0,215000.0,3,2,0.19,Chicopee,1020,1828.0,117.614880,Hampden County,MA,30704.0,951.4,0.925140,106.769095,105.789421,550.0,828125.0
1,389900.0,3,1,24.00,Northampton,1062,1559.0,250.096216,Hampshire County,MA,9835.0,217.6,3.630387,122.235452,124.761768,191.0,639133.0
2,269900.0,2,1,0.46,South Hadley,1075,1312.0,205.716463,Hampshire County,MA,18051.0,393.3,2.910859,112.181953,115.181126,297.0,697896.0
3,314900.0,5,2,0.28,Chicopee,1013,2219.0,141.910771,Hampden County,MA,22709.0,1549.1,0.062286,109.114246,104.576258,317.0,516431.0
4,244999.0,4,1,0.23,Springfield,1104,1285.0,190.660700,Hampden County,MA,23226.0,1714.0,0.645911,104.785728,101.063511,528.0,453300.0
5,199000.0,2,2,0.85,Deerfield,1373,1664.0,119.591346,Franklin County,MA,5076.0,85.0,2.294487,126.845728,139.556753,139.0,224315.0
6,279000.0,6,3,0.13,Chicopee,1020,3348.0,83.333333,Hampden County,MA,30704.0,951.4,0.925140,106.769095,105.789421,550.0,828125.0
7,589000.0,4,3,1.60,Ludlow,1056,2964.0,198.717949,Hampden County,MA,21050.0,298.9,2.608102,99.921771,105.057502,412.0,747067.0
8,399900.0,3,3,0.35,Easthampton,1027,1380.0,289.782609,Hampshire County,MA,17708.0,168.5,9.873599,125.949999,124.367513,372.0,671249.0
9,359000.0,3,2,1.03,Westfield,1085,1670.0,214.970060,Hampden County,MA,41715.0,269.7,2.075971,118.450463,101.617250,845.0,1371042.0


In [59]:
real_estate_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451719 entries, 0 to 451718
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   price                  451719 non-null  float64
 1   bed                    451719 non-null  int64  
 2   bath                   451719 non-null  int64  
 3   acre_lot               451719 non-null  float64
 4   city                   451719 non-null  object 
 5   zip_code               451719 non-null  int64  
 6   house_size             451719 non-null  float64
 7   price_per_sqft         451719 non-null  float64
 8   county                 451719 non-null  object 
 9   state                  451719 non-null  object 
 10  population             451719 non-null  float64
 11  density                451719 non-null  float64
 12  dist_highway           451719 non-null  float64
 13  dist2_large_airport    451719 non-null  float64
 14  dist_to_shore          451719 non-nu

In [60]:
real_estate_features_df.describe()

,price,bed,bath,acre_lot,zip_code,house_size,price_per_sqft,population,density,dist_highway,dist2_large_airport,dist_to_shore,number_of_business,adjusted_gross_income
count,4.517190e+05,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,4.517190e+05
mean,5.518166e+05,3.284365,2.428187,11.148105,59483.667176,1943.094021,285.082448,33545.607781,1265.905413,6.976981,76.397552,176.731627,691.471818,1.374152e+06
std,7.508133e+05,1.315466,1.195087,774.598407,31654.732566,1763.175229,240.130182,19403.167687,1784.271593,14.525969,101.157852,235.796885,513.583607,1.163807e+06
min,1.000000e+00,1.000000,1.000000,0.000000,1001.000000,100.000000,0.000282,0.000000,0.000000,0.000283,0.424420,0.011449,1.000000,2.925000e+03
25%,2.499500e+05,3.000000,2.000000,0.130000,31320.000000,1303.000000,150.142993,19787.000000,244.800000,1.365326,19.630244,18.578272,319.000000,6.152530e+05
50%,3.950000e+05,3.000000,2.000000,0.190000,70601.000000,1714.000000,214.863498,31480.000000,811.700000,2.912591,36.311981,65.523818,585.000000,1.108688e+06
75%,6.250000e+05,4.000000,3.000000,0.320000,91106.000000,2300.000000,333.236996,44608.000000,1707.000000,6.403780,90.491868,270.690594,943.000000,1.785317e+06
max,8.000000e+07,444.000000,222.000000,100000.000000,99402.000000,927828.000000,15015.015015,130352.000000,58289.600000,299.940968,994.951376,1330.442647,7263.000000,1.522467e+07


In [61]:
real_estate_features_df.describe(include = 'object')

,city,county,state
count,451719,451719,451719
unique,8038,1243,49
top,Houston,Los Angeles County,CA
freq,10570,20835,93286


Import Library


In [62]:
import pandas as pd
import numpy as np
import seaborn as sns
from string import ascii_letters
import matplotlib.pyplot as plt
import datetime as dt
import requests
from lxml import html
import math
import json
import re
import os

# 1. County Demographics Data Cleaning and Wrangling


## 1.1 Loading County Demogrphics Dataset

In [63]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("glozab/county-level-us-demographic-data-1990-2020")

print("Path to dataset files:", path)

100%|██████████| 13.1M/13.1M [00:01<00:00, 10.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/glozab/county-level-us-demographic-data-1990-2020/versions/1


In [64]:
files = os.listdir(path)
print("Files in dataset:", files)

Files in dataset: ['county_demographics.csv']


In [65]:
file_path = os.path.join(path, 'county_demographics.csv')
county_demographics_df = pd.read_csv(file_path)

print(county_demographics_df.head())

   year  fips  population  w_population  b_population  o_population  \
0  1990  1025       27289         15579         11643            35   
1  1990  1031       40293         32869          6950           160   
2  1990  1041       13598         10068          3516            11   
3  1990  1053       35526         24377         10050          1045   
4  1990  1101      209537        119702         87856           415   

   nh_population  hi_population  na_population  male_population  ...  \
0          27196             93              0            13052  ...   
1          39831            462              0            19673  ...   
2          13576             22              0             6421  ...   
3          35378            148              0            17454  ...   
4         207933           1604              0            98854  ...   

   age9_population_ratio  age10_population_ratio  age11_population_ratio  \
0                0.06263                 0.05552                

## 1.2 County Demographics Dataset Cleaning

In [66]:
county_demographics_df.dtypes

,0
year,int64
fips,int64
population,int64
w_population,int64
b_population,int64
o_population,int64
nh_population,int64
hi_population,int64
na_population,int64
male_population,int64


In [67]:
print(county_demographics_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97287 entries, 0 to 97286
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97287 non-null  int64  
 1   fips                     97287 non-null  int64  
 2   population               97287 non-null  int64  
 3   w_population             97287 non-null  int64  
 4   b_population             97287 non-null  int64  
 5   o_population             97287 non-null  int64  
 6   nh_population            97287 non-null  int64  
 7   hi_population            97287 non-null  int64  
 8   na_population            97287 non-null  int64  
 9   male_population          97287 non-null  int64  
 10  female_population        97287 non-null  int64  
 11  age0_population          97287 non-null  int64  
 12  age1_population          97287 non-null  int64  
 13  age2_population          97287 non-null  int64  
 14  age3_population       

In [68]:
print(county_demographics_df.isna().sum().sort_values(ascending=False))

year                       0
fips                       0
population                 0
w_population               0
b_population               0
o_population               0
nh_population              0
hi_population              0
na_population              0
male_population            0
female_population          0
age0_population            0
age1_population            0
age2_population            0
age3_population            0
age4_population            0
age5_population            0
age6_population            0
age7_population            0
age8_population            0
age9_population            0
age10_population           0
age11_population           0
age12_population           0
age13_population           0
age14_population           0
age15_population           0
age16_population           0
age17_population           0
age18_population           0
w_population_ratio         0
b_population_ratio         0
o_population_ratio         0
nh_population_ratio        0
hi_population_

### Rename Columns for Consistency

In [69]:
county_demographics_df.columns = county_demographics_df.columns.str.strip().str.lower().str.replace(' ', '_')

 ### Handle Duplicates

In [70]:
county_demographics_df = county_demographics_df.drop_duplicates()
county_demographics_df = county_demographics_df.dropna()
county_demographics_df.head()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age9_population_ratio,age10_population_ratio,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.06263,0.05552,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.07455,0.06031,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.06582,0.05354,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.06820,0.05694,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.06966,0.05219,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145


In [71]:
county_demographics_df.describe()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age9_population_ratio,age10_population_ratio,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio
count,97287.000000,97287.000000,9.728700e+04,9.728700e+04,9.728700e+04,97287.000000,9.728700e+04,9.728700e+04,97287.0,9.728700e+04,...,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000
mean,2005.009436,30412.859950,9.365957e+04,7.529841e+04,1.252406e+04,1129.919044,8.009754e+04,1.356203e+04,0.0,4.598061e+04,...,0.068098,0.067555,0.064839,0.060595,0.054948,0.048050,0.039687,0.030821,0.021798,0.020099
std,8.942825,15147.842995,3.034699e+05,2.259523e+05,5.716518e+04,4815.437070,2.171562e+05,1.058393e+05,0.0,1.489240e+05,...,0.011292,0.010417,0.011904,0.014153,0.014746,0.013994,0.011871,0.009476,0.007503,0.008808
min,1990.000000,1001.000000,5.500000e+01,2.900000e+01,0.000000e+00,0.000000,4.900000e+01,0.000000e+00,0.0,2.900000e+01,...,0.008400,0.013420,0.009150,0.007710,0.005320,0.004840,0.002570,0.000000,0.000000,0.000000
25%,1997.000000,18183.000000,1.094800e+04,9.225500e+03,9.500000e+01,40.000000,1.011600e+04,1.710000e+02,0.0,5.445000e+03,...,0.059970,0.060400,0.055950,0.048960,0.044160,0.038620,0.032010,0.024680,0.016890,0.014210
50%,2005.000000,29179.000000,2.490200e+04,2.166100e+04,8.000000e+02,136.000000,2.327000e+04,6.350000e+02,0.0,1.231900e+04,...,0.068510,0.067780,0.065460,0.060580,0.052770,0.046150,0.038470,0.029950,0.021000,0.018590
75%,2013.000000,45083.000000,6.347900e+04,5.624700e+04,5.440000e+03,550.000000,5.989000e+04,3.148000e+03,0.0,3.135050e+04,...,0.075770,0.074500,0.073120,0.071100,0.064605,0.055630,0.045940,0.035970,0.025670,0.024350
max,2020.000000,99999.000000,1.009486e+07,7.301940e+06,1.437609e+06,172454.000000,5.508975e+06,4.868179e+06,0.0,4.975410e+06,...,0.160000,0.155960,0.153010,0.258060,0.181820,0.183910,0.183580,0.143400,0.144440,0.195400


## 1.2 Loading Fips Code DataSet




In [ ]:
from google.colab import files
uploaded = files.upload()
fips_df = pd.read_csv('state_and_county_fips_master.csv')
fips_df.head()

In [ ]:
fips_df.info()

## 1.3 Merging the county_demographics_df with fips_df

In [ ]:
county_demographics_merged_df = county_demographics_df.merge(
    fips_df,
    on='fips',
    how='left'
)
county_demographics_merged_df.head()

### Sanity Check After Merging

In [ ]:
print(county_demographics_merged_df.columns)         # See all columns
print(county_demographics_merged_df[['fips', 'name']].head())  # Confirm correct matches
print(county_demographics_merged_df.isna().sum())    # Check for unmatched FIPS

## 1.4 Cleaning the Merged Data

Checking for unmatched FIPS — that output tells us 174 rows in the county_demographics_df didn't find a match in fips_df, because name, state, and any other info from fips_df came back as NaN.

In [ ]:
unmatched = county_demographics_merged_df[county_demographics_merged_df['name'].isna()]
print(unmatched['fips'].unique()[:10])

In [ ]:
county_demographics_merged_df.info()

Drop the unmatched rows.

In [ ]:
county_demographics_merged_df = county_demographics_merged_df[county_demographics_merged_df['name'].notna()].reset_index(drop=True)
county_demographics_merged_df.info()


In [ ]:
county_demographics_merged_df = county_demographics_merged_df.rename(columns={'name': 'county'})
county_demographics_merged_df = county_demographics_merged_df.dropna()
county_demographics_merged_df = county_demographics_merged_df.drop_duplicates()
county_demographics_merged_df.info()

In [ ]:
county_demographics_merged_df.head()

# 2. FEMA Disaster Declarations Data Cleaning and Wrangling

In this section, we clean and wrangle FEMA disaster declaration data to prepare it for analysis. The raw data is filtered to include only disasters from the past 10 years, joined with ZIP codes using a county-level crosswalk, and aggregated to generate a ZIP-level disaster count feature for downstream housing price modeling.

The final output table, `zip_disaster_counts`, contains the number of FEMA disaster declarations per ZIP code over the last decade. This will be used as a static risk feature in our housing price model.





##2.1 Import Required Libraries



In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests

## 2.2 Load FEMA Disaster Declarations Data

Note: We loaded the full dataset here to understand the available columns. In the next step, we’ll select only the relevant ones.

In [ ]:
fema_url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.csv"
fema_file_path = "DisasterDeclarationsSummaries.csv"

# Download the file if it doesn't already exist
if not os.path.exists(fema_file_path):
    response = requests.get(fema_url)
    with open(fema_file_path, "wb") as file:
        file.write(response.content)

# Load the entire dataset first
fema_raw_df = pd.read_csv(fema_file_path)

# Preview full column names
fema_raw_df.head()


## 2.3 Select Relevant Columns：

In [ ]:
# Select only the columns we need
fema_df = fema_raw_df[[
    'disasterNumber', 'state', 'designatedArea', 'declarationDate',
    'fipsStateCode', 'fipsCountyCode'
]].copy()

# Convert to datetime
fema_df['declarationDate'] = pd.to_datetime(fema_df['declarationDate'])

# Preview cleaned dataframe
fema_df.head()


## 2.4 Filter Data from the Last 10 Years

In [ ]:
from pandas.tseries.offsets import DateOffset

# Today's date and dynamic 10-year cutoff
today = pd.Timestamp.now(tz='UTC')
cutoff_date = today - DateOffset(years=10)

# Filter FEMA data
fema_recent_df = fema_df[fema_df['declarationDate'] >= cutoff_date].copy()

# Check the date range
print("Earliest disaster:", fema_recent_df['declarationDate'].min())
print("Latest disaster:", fema_recent_df['declarationDate'].max())
print("Total records:", fema_recent_df.shape[0])

# Preview
fema_recent_df.head()

## 2.5 Clean FEMA Data (Standardize & Prepare for Join)

In [ ]:
# Construct full 5-digit county-level FIPS code
fema_recent_df['fips'] = (
    fema_recent_df['fipsStateCode'].astype(str).str.zfill(2) +
    fema_recent_df['fipsCountyCode'].astype(str).str.zfill(3)
)

# Preview
fema_recent_df.head()

## 2.6 Load County-to-ZIP Mapping Table from Kaggle

In [ ]:
# Install kagglehub if not already installed
!pip install kagglehub --quiet

import kagglehub
import os
import pandas as pd

# Download dataset from Kaggle
path = kagglehub.dataset_download("danofer/zipcodes-county-fips-crosswalk")

print("Path to dataset files:", path)

# List files
files = os.listdir(path)
print("Files in dataset:", files)

# Load the crosswalk CSV
file_path = os.path.join(path, "ZIP-COUNTY-FIPS_2017-06.csv")
zip_crosswalk_df = pd.read_csv(file_path)

zip_crosswalk_df['fips'] = zip_crosswalk_df['STCOUNTYFP'].astype(str).str.zfill(5)


# Preview
zip_crosswalk_df.head()



## 2.7 Merge FEMA with ZIP (via FIPS)

In [ ]:
fema_with_zip = fema_recent_df.merge(
    zip_crosswalk_df[['ZIP', 'fips']],
    on='fips',
    how='left'
)

# Drop FEMA rows with no ZIP match
#fema_with_zip = fema_with_zip.dropna(subset=['ZIP'])

# Preveiw
fema_with_zip.head()

In [ ]:
print(f"Total FEMA records after 10-year filter: {fema_recent_df.shape[0]}")
print(f"FEMA records after ZIP merge: {fema_with_zip.shape[0]}")
print(f"Number of FEMA records with ZIP: {fema_with_zip['ZIP'].notna().sum()}")
print(f"Number of FEMA records missing ZIP: {fema_with_zip['ZIP'].isna().sum()}")


## 2.8: Merge Validation — FEMA and ZIP Mapping

In this section, we validate whether FEMA disaster records were successfully joined with ZIP codes using fips codes. We check the match rate, ZIP assignment distribution, and inspect records with missing ZIPs.



### 2.8.1 Match Rate Check
This checks whether the merge was successful for a significant portion of the data.

In [ ]:
total_records = fema_with_zip.shape[0]
matched_zip_count = fema_with_zip['ZIP'].notna().sum()
missing_zip_count = fema_with_zip['ZIP'].isna().sum()
match_rate = matched_zip_count / total_records

print(f"Total FEMA records (10 yrs):     {total_records}")
print(f"Matched ZIPs:                    {matched_zip_count}")
print(f"Missing ZIPs:                    {missing_zip_count}")
print(f"ZIP Match Rate:                  {match_rate:.2%}")


### 2.8.2 ZIP Duplication per FIPS

One county (FIPS) may correspond to multiple ZIP codes. We inspect the distribution to ensure the ZIP mapping is reasonable.


In [ ]:
zip_per_fips = (
    fema_with_zip[['fips', 'ZIP']]
    .drop_duplicates()
    .groupby('fips')
    .size()
    .reset_index(name='zip_count')
)

zip_per_fips['zip_count'].describe()


###2.8.3 Missing ZIPs (Rows That Didn't Match)
Identify FEMA records from counties not covered in the ZIP mapping dataset.

In [ ]:
missing_zip_df = fema_with_zip[fema_with_zip['ZIP'].isna()]
print(f"Number of FEMA records without ZIP after merge: {missing_zip_df.shape[0]}")
missing_zip_df[['state', 'designatedArea', 'fips']].drop_duplicates().head(10)


#### 2.8.3 ZIP Missing Analysis

After merging FEMA disaster data with ZIP codes using FIPS codes, **934** records could not be matched.

Top unmatched designatedArea examples:
*   Resighini Rancheria (Indian Reservation), CA
*   Nez Perce Indian Reservation, ID
*   Yakama Reservation, WA
*   Warm Springs Indian Reservation, OR
*   Crow Indian Reservation, MT
*   San Carlos Indian Reservation, AZ
*   Hopland Rancheria (Indian Reservation), CA



**Possible reasons:**
- Indian Reservations and tribal lands often don't align with standard county or ZIP boundaries

- Non-standard area names that don’t match ZIP datasets (e.g., Rancheria, Reservation)

**Action:** These records can be excluded


In [ ]:
# Drop FEMA records with missing ZIPs
fema_with_zip['zip_matched'] = fema_with_zip['ZIP'].notna()
fema_matched_df = fema_with_zip[fema_with_zip['zip_matched']].copy().reset_index(drop=True)

# Preview
print(fema_matched_df.shape)
fema_matched_df.head()

###2.8.4 FEMA-ZIP Merge Summary

In [ ]:
# Total rows
rows_before = fema_with_zip.shape[0]
rows_after = fema_matched_df.shape[0]

# ZIP matching stats
matched_rows = fema_with_zip['ZIP'].notna().sum()
missing_rows = fema_with_zip['ZIP'].isna().sum()
match_rate = matched_rows / rows_before
final_match_rate = 1.0  # because all rows in fema_matched_df have ZIPs

# ZIPs per FIPS (before filtering)
zip_per_fips = (
    fema_with_zip[['fips', 'ZIP']]
    .drop_duplicates()
    .groupby('fips')
    .size()
    .reset_index(name='zip_count')
)

# Summary stats
print("FEMA-ZIP Merge Summary:")
print(f"Total FEMA records (before filtering):   {rows_before}")
print(f"Total FEMA records (after filtering):    {rows_after}")
print(f"Matched ZIPs:                             {matched_rows}")
print(f"Missing ZIPs:                             {missing_rows}")
print(f"ZIP Match Rate:                           {match_rate:.2%}")
print(f"ZIP Match Rate (final dataset):           {final_match_rate:.2%}")
print(f"Unique county FIPS:                       {zip_per_fips.shape[0]}")
print(f"Median ZIPs per county:                   {zip_per_fips['zip_count'].median():.0f}")
print(f"Max ZIPs per county:                      {zip_per_fips['zip_count'].max()}")


##2.10 Disaster Count Aggregation by ZIP (Last 10 Years)

To support downstream analysis, we compute the number of FEMA disaster declarations in the past 10 years for each ZIP code. This gives a static risk exposure feature per ZIP that can be merged with housing data.

The aggregation is based on the cleaned FEMA dataset `fema_matched_df`, which includes only matched ZIP-level records in past 10 years.

In [ ]:
# Count the number of disasters per ZIP code
zip_disaster_counts = (
    fema_matched_df.groupby('ZIP')['disasterNumber']
    .nunique()
    .reset_index(name='disaster_count_10yrs')
)

# Format ZIPs as 5-digit strings
# zip_disaster_counts['ZIP'] = zip_disaster_counts['ZIP'].astype(int).astype(str).str.zfill(5)

# Make sure ZIP is stored as integer (drop decimals)
zip_disaster_counts['ZIP'] = zip_disaster_counts['ZIP'].astype(int)

# Reset index for cleanliness
zip_disaster_counts = zip_disaster_counts.reset_index(drop=True)

# Preview top ZIPs by disaster count
zip_disaster_counts.sort_values(by='disaster_count_10yrs', ascending=False).head(10)


## Summary of Disaster Count Aggregation

The resulting table `zip_disaster_counts` contains the number of unique FEMA disaster declarations per ZIP code over the past 10 years. Each row represents a ZIP and its corresponding disaster exposure:

- **ZIP**: 5-digit ZIP code (int)
- **disaster_count_10yrs**: Total number of FEMA disaster declarations in that ZIP over the past 10 years

This table will serve as a **static risk feature** that can be joined with the main housing dataset using the ZIP code as key.


# HPI Dataframe by Zip

## Uploading Data

In [ ]:
file_path = '/content/drive/MyDrive/hpi_at_bdl_zip5.csv'
# Read the file
hpi_df = pd.read_csv(file_path)
hpi_df.info()

In [ ]:
# Drop meaningless columns
columns_to_drop = ["HPI with 2000 base", "Unnamed: 6","Unnamed: 7","Unnamed: 8","Unnamed: 9","Unnamed: 10","Unnamed: 11","Unnamed: 12",]
hpi_df = hpi_df.drop(columns=columns_to_drop)

# Convert "Five-Digit ZIP Code" to string and rename the column to zip_code
hpi_df["Five-Digit ZIP Code"] = hpi_df["Five-Digit ZIP Code"].apply(lambda x: f"{int(x):05}")
hpi_df = hpi_df.rename(columns={"Five-Digit ZIP Code": "zip_code"})

# Filter timeline from 1990-2020
filtered_hpi_df = hpi_df[(hpi_df['Year'] >= 1990) & (hpi_df['Year'] <= 2020)]

# Rename columns
filtered_hpi_df = filtered_hpi_df.rename(columns={"Annual Change (%)": "annual_change"})
filtered_hpi_df = filtered_hpi_df.rename(columns={"HPI with 1990 base": "hpi_1990"})


filtered_hpi_df = filtered_hpi_df.reset_index(drop=True)
filtered_hpi_df.info()

In [ ]:
filtered_hpi_df.head(10)

## Check Data Quality for Housing Price Time Series

In [ ]:
# Check the number of rows in dataframe
print("Rows in filtered_hpi_df before dropping NA:\n", len(filtered_hpi_df))
print("\n")

# Check Nulls or missing values
missing_values = filtered_hpi_df.isna().sum()
missing_percentage = (missing_values / len(filtered_hpi_df)) * 100

print("Missing value breakdown:\n", missing_values)
print("\n")
print("Missing Percentage:\n", missing_percentage)
print("\n")
# Find rows where only 'HPI' is NaN and other columns have non-NaN values
only_hpi_na = filtered_hpi_df[filtered_hpi_df['HPI'].isna() & filtered_hpi_df.drop(columns=['HPI']).notna().all(axis=1)]
# Display how many rows have only 'HPI' NaN and not NaN in other columns
print(f"Rows with NaN in 'HPI' but not in other columns: {only_hpi_na.shape[0]}")

In [ ]:
# Drop hpi_1990 column since there is a large amount of missing value and won't be good reference data point
filtered_hpi_df = filtered_hpi_df.drop(columns=['hpi_1990'])
filtered_hpi_df.info()

In [ ]:
# Drop NAs in HPI and using HPI as major housing price prediction
# // Potetial Alternative Treatment on Missing values is to get Average of HPI for ZIPs if they are in the same MSA or County

cleaned_hpi_df = filtered_hpi_df.dropna(subset=['annual_change', 'HPI'], how='all')
print("Rows in cleaned_hpi_df after dropping NA:\n", len(cleaned_hpi_df))

cleaned_hpi_df.head()

## Transform Data for Zip code level with annual HPI info

In [ ]:
# Convert the HPI table by Zip level with annual HPI info appended for each year


# Pivot the data: reshape so each year’s values become their own set of columns
pivoted_hpi_df = filtered_hpi_df.pivot_table(
    index='zip_code',
    columns='Year',
    values=['HPI', 'annual_change']
)

# Flatten the multi-level column index
pivoted_hpi_df.columns = [f'{metric}_{year}' for metric, year in pivoted_hpi_df.columns]

# Reset index to turn zip_code back into a column
pivoted_hpi_df = pivoted_hpi_df.reset_index()

# Display the result
print(pivoted_hpi_df.head())
print("\n")
print("Rows in pivoted_hpi_df", len(pivoted_hpi_df))

In [ ]:
# merged_hpi_df = pd.merge(merged_zip_county_MSA_df, pivoted_hpi_df, left_on='zip_code', right_on='zip_code', how='inner')

merged_hpi_df = pd.merge(
    merged_zip_county_MSA_df,
    pivoted_hpi_df,
    left_on='zip_code',
    right_on='zip_code',
    how='outer',  # use 'outer' to catch all unmatched rows from both sides
    indicator=True
)

In [ ]:
# Rows only from left DataFrame (no match from pivoted_hpi_df)
unmatched_left = merged_hpi_df[merged_hpi_df['_merge'] == 'left_only']

# Rows only from right DataFrame (no match from merged_zip_county_MSA_df)
unmatched_right = merged_hpi_df[merged_hpi_df['_merge'] == 'right_only']

# Zip codes from left DataFrame that had no match in right
print("Unmatched zip codes from left DataFrame:")
print(unmatched_left['zip_code'].unique())

# Zip codes from right DataFrame that had no match in left
print("Unmatched zip codes from right DataFrame:")
print(unmatched_right['zip_code'].unique())
print("\n")

missing_from_right = (merged_hpi_df['_merge'] == 'left_only').sum()
missing_from_left = (merged_hpi_df['_merge'] == 'right_only').sum()
print(f"Zip codes missing from pivoted_hpi_df (right): {missing_from_right}")
print(f"Zip codes missing from merged_zip_county_MSA_df (left): {missing_from_left}")

# merged_hpi_df.drop(columns=['_merge'], inplace=True)

In [ ]:
print(merged_hpi_df.head())
print("\n")
print("Rows in pivoted_hpi_df", len(pivoted_hpi_df))

## Merging HPI time series with Demographic and FEMA dataframe

In [ ]:
# Merge HPI with Demographic and FEMA dataframe

